# Data Gathering

In [1]:
#imports

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests 

In [2]:
#create an empty list to collect all reviews
reviews  = []

#create an empty list to collect rating stars
stars = []

#create an empty list to collect country the reviewer is from
country = []

dateflown = []

In [3]:
for i in range(1, 38):
    page = requests.get(f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100")
    
    
    soup = BeautifulSoup(page.content, "html5")
    
    #reviews
    for item in soup.find_all("div", class_="text_content"):
        reviews.append(item.text)
    
    #stars
    for item in soup.find_all("div", class_ = "rating-10"):
        try:
            stars.append(item.span.text)
        except:
            print(f"Error on page {i}")
            stars.append("None")
            
    #stars       
    for sibling in soup.find_all('td', class_='review-rating-header date_flown'):
        try:
            dateflown.append(sibling.find_next_sibling('td').text.strip())
        except AttributeError:
            print(f"Error on page {i}")
            dateflown.append("None")
            
        
    #country
    for item in soup.find_all("h3"):
        country.append(item.span.next_sibling.text.strip(" ()"))

Error on page 33
Error on page 34
Error on page 34
Error on page 36
Error on page 37


In [4]:
reviews = reviews[:3007]

In [5]:
country = country[:3007]

In [6]:
stars = stars[:3007]

In [7]:
df = pd.DataFrame({"reviews":reviews,"stars": stars, "dateflown":  dateflown, "country": country})

In [8]:
df.head()


,reviews,stars,dateflown,country
0,✅ Trip Verified | We chose Rotterdam and Lond...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,March 2024,Netherlands
1,✅ Trip Verified | The entire booking and ticke...,6,April 2024,United States
2,Not Verified | British Airways cancelled my ...,1,January 2024,United States
3,Not Verified | I wanted to write this review s...,1,March 2024,United States
4,✅ Trip Verified | Check in fast and friendly....,10,March 2024,United Kingdom


# Data Cleaning

In [9]:
df.describe()

,reviews,stars,dateflown,country
count,3007,3007,3007,3007
unique,3005,11,117,69
top,British Airways from Tampa to Gatwick on Boein...,1,August 2015,United Kingdom
freq,2,795,83,1856


In [10]:
df['reviews'] = df['reviews'].str.replace('✅ Trip Verified | ', '', regex=False).str.replace('Not Verified | ', '', regex=False)


In [18]:
df

,reviews,stars,dateflown,country
0,We chose Rotterdam and London City airport be...,5,Mar/2024,Netherlands
1,The entire booking and ticketing experience ha...,6,Apr/2024,United States
2,British Airways cancelled my flight less tha...,1,Jan/2024,United States
3,I wanted to write this review so I could give ...,1,Mar/2024,United States
4,Check in fast and friendly. Security a breeze...,10,Mar/2024,United Kingdom
...,...,...,...,...
3002,We were boarded quickly but suffered a weather...,2,May/2015,United Kingdom
3003,Terrible. Flight time is well over 2 hours. Se...,6,May/2015,United Kingdom
3004,"28th May 2015, LHR-DEN, B747, Seat 29A. Paid ...",1,Jun/2015,United Kingdom
3005,"9/6/15, ORD-LHR, 777-300, Club World, Seat 13b...",8,Jun/2015,United Kingdom


In [12]:
 df['stars'].unique()

array(['\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '6', '1', '10', '7', '3', '4',
       '8', '5', '9', '2'], dtype=object)

In [13]:
df['stars'] = df['stars'].str.replace('\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '5', regex=False)

In [15]:
df['dateflown'] = pd.to_datetime(df['dateflown'])  # Ensure it's in datetime format
df['dateflown'] = df['dateflown'].dt.strftime('%b/%Y')  # Convert to "Month/Year" format


C:\Users\pearl\AppData\Local\Temp\ipykernel_25552\585443733.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['dateflown'] = pd.to_datetime(df['dateflown'])  # Ensure it's in datetime format


In [16]:
df.dtypes

reviews      object
stars        object
dateflown    object
country      object
dtype: object

In [17]:
df.isnull().values.any()


False

In [19]:
df.to_csv('britishthing.csv', index=False)
